# Maps SARS-CoV-2 Mutations to 3D Protein Structures
[Work in progress]

This notebook maps mutation frequency of SARS-CoV-2 strains onto 3D protein structures with bound antibody fragments in the [Protein Data Bank](https://www.wwpdb.org/). SARS-CoV2-2 strains and their mutations have been aggregated in the [COVID-19-Net Knowledge Graph](https://github.com/covid-19-net/covid-19-community).

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm as cm
import ipywidgets as widgets
from ipywidgets import interact, IntSlider, FloatSlider, SelectMultiple
from py2neo import Graph
import py3Dmol

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
output_file_name = 'mutations3d.csv' # mutations mapped to 3D protein structures

#### Connect to COVID-19-Community Knowledge Graph
[COVID-19-Net Knowledge Graph](https://github.com/covid-19-net/covid-19-community)

In [4]:
graph = Graph("bolt://132.249.238.185:7687", user="reader", password="demo")

### Taxonomy ids for pathogen (SARS-CoV-2) and host (human)

In [5]:
pathogen_taxonomy_id = 'taxonomy:2697049'
host_taxonomy_id = 'taxonomy:9606'
with_antibodies = True

### Get list of SARS-CoV-2 proteins

In [6]:
query = """
MATCH (p:Protein{taxonomyId: $pathogen_taxonomy_id})-[t:HAS_TERTIARY_STRUCTURE]->(:Chain)-[:IS_PART_OF_STRUCTURE]->(s:Structure)
WHERE t.coverage > 0.2 // eliminate polyprotein
RETURN p.name AS protein, p.accession as accession, p.proId as proId, p.sequence as sequence, s.description AS description
ORDER BY protein
"""

In [7]:
structures = graph.run(query, pathogen_taxonomy_id=pathogen_taxonomy_id).to_data_frame()

In [8]:
proteins = structures[['protein', 'accession', 'proId', 'sequence']].drop_duplicates()

In [9]:
proteins.head(50)

,protein,accession,proId,sequence
0,2'-O-methyltransferase,uniprot:P0DTD1,uniprot.chain:PRO_0000449633,SSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMNVAK...
26,3C-like proteinase,uniprot:P0DTC1,uniprot.chain:PRO_0000449639,SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTS...
402,Envelope small membrane protein,uniprot:P0DTC4,uniprot.chain:PRO_0000449651,MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNI...
407,Helicase,uniprot:P0DTD1,uniprot.chain:PRO_0000449630,AVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVN...
532,Host translation inhibitor nsp1,uniprot:P0DTC1,uniprot.chain:PRO_0000449635,MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHL...
534,Non-structural protein 10,uniprot:P0DTC1,uniprot.chain:PRO_0000449644,AGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVKMLC...
563,Non-structural protein 7,uniprot:P0DTC1,uniprot.chain:PRO_0000449641,SKMSDVKCTSVVLLSVLQQLRVESSSKLWAQCVQLHNDILLAKDTT...
602,Non-structural protein 8,uniprot:P0DTC1,uniprot.chain:PRO_0000449642,AIASEFSSLPSYAAFATAQEAYEQAVANGDSEVVLKKLKKSLNVAK...
656,Non-structural protein 9,uniprot:P0DTC1,uniprot.chain:PRO_0000449643,NNELSPVALRQMSCAAGTTQTACTDDNALAYYNTTKGGRFVLALLS...
664,Nucleoprotein,uniprot:P0DTC9,uniprot.chain:PRO_0000449656,MSDNGPQNQRNAPRITFGGPSDSTGSNQNGERSGARSKQRRPQGLP...


In [10]:
protein_list = structures['protein'].unique()

In [11]:
protein_widget = widgets.Dropdown(options=protein_list, description='Select protein:', value='Spike glycoprotein',
                                  style={'description_width': 'initial'}, )

### Select SARS-CoV-2 Protein

In [12]:
display(protein_widget)

Dropdown(description='Select protein:', index=15, options=("2'-O-methyltransferase", '3C-like proteinase', 'En…

In [13]:
protein_name = protein_widget.value
print('Protein name:', protein_name)

Protein name: Spike glycoprotein


In [14]:
subset = structures.query(f'protein == "{protein_name}"').copy()
subset.fillna('', inplace=True)

In [15]:
subset.head()

,accession,description,proId,protein,sequence
751,uniprot:P0DTC2,"Spike glycoprotein, 2-acetamido-2-deoxy-beta-D...",,Spike glycoprotein,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
752,uniprot:P0DTC2,"Spike glycoprotein, P008_056 Fab Heavy chain, ...",,Spike glycoprotein,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
753,uniprot:P0DTC2,"Spike glycoprotein, 2-acetamido-2-deoxy-beta-D...",,Spike glycoprotein,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
754,uniprot:P0DTC2,"Spike glycoprotein, P008_056 Fab Heavy chain, ...",,Spike glycoprotein,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
755,uniprot:P0DTC2,"Spike glycoprotein, P008_056 Fab Heavy chain, ...",,Spike glycoprotein,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


### Get total number of strains

In [16]:
query = """
MATCH (s:Strain)
WHERE s.taxonomyId = $pathogen_taxonomy_id AND s.hostTaxonomyId = $host_taxonomy_id
RETURN count(s)
"""

In [17]:
strains = graph.evaluate(query, pathogen_taxonomy_id=pathogen_taxonomy_id, 
                         host_taxonomy_id=host_taxonomy_id)

In [18]:
print('Total number of human strains:', strains)

Total number of human strains: 912224


### Get variants for selected protein

In [19]:
query = """
MATCH (p:Protein{reviewed: True})-[:HAS_VARIANT]->(v:Variant{variantConsequence:'missense_variant'})<-[:HAS_VARIANT]-(s:Strain)
WHERE p.name = $protein_name AND p.taxonomyId = $pathogen_taxonomy_id AND s.hostTaxonomyId = $host_taxonomy_id
WITH v.proteinPosition AS residue, count(v.proteinVariant) AS count, 
     v.proteinVariant + '(' + count(v.proteinVariant) + ')' AS variationId ,
     split(v.proteinVariant, ':')[1] + '(' + count(v.proteinVariant) + ')' AS annotation
     ORDER by count DESC
WITH residue, count, variationId, annotation
RETURN residue, collect(variationId) AS variationId, collect(annotation) AS annotation, sum(count) AS count ORDER BY residue
"""

### Add mutation annotation to each residue

In [20]:
variants = graph.run(query, protein_name=protein_name, pathogen_taxonomy_id=pathogen_taxonomy_id, 
                     host_taxonomy_id=host_taxonomy_id).to_data_frame()

In [21]:
variants.shape

(1199, 4)

In [22]:
variants.head()

,annotation,count,residue,variationId
0,"[p.2F>L(67), p.2F>V(9), p.2F>S(4), p.2F>Y(2)]",82,2,"[S:p.2F>L(67), S:p.2F>V(9), S:p.2F>S(4), QHD43..."
1,"[p.3V>G(389), p.3V>I(18), p.3V>F(7)]",414,3,"[S:p.3V>G(389), S:p.3V>I(18), S:p.3V>F(7)]"
2,"[p.4F>S(6), p.4F>L(5), p.4F>I(1)]",12,4,"[S:p.4F>S(6), S:p.4F>L(5), S:p.4F>I(1)]"
3,"[p.5L>F(13770), p.5L>I(4), p.5L>V(4)]",13778,5,"[S:p.5L>F(13770), S:p.5L>I(4), S:p.5L>V(4)]"
4,"[p.6V>A(367), p.6V>F(314), p.6V>I(41), p.6V>L(...",728,6,"[S:p.6V>A(367), S:p.6V>F(314), S:p.6V>I(41), S..."


In [23]:
variants.sort_values('count', ascending=False, inplace=True)

In [24]:
variants['variationId'] = variants['variationId'].apply(lambda x: ', '.join(x))
variants['annotation'] = variants['annotation'].apply(lambda x: ', '.join(x))

In [25]:
variants['annotation'] = variants['annotation'].str.replace('p.', '')

Create a color scale based on the log mutation frequency

In [26]:
total = variants['count'].sum()
variants['scale'] = variants['count'].apply(np.log10) / math.log10(total)

In [27]:
n_colors = 100
colors = cm.Reds(np.linspace(0.0, 1.0, n_colors))
col = np.empty(n_colors, dtype=object)

for i, color in enumerate(colors):
    col[i] = matplotlib.colors.rgb2hex(color)

In [28]:
variants['color'] = variants['scale'].apply(lambda x: col[round(x*n_colors)])

In [29]:
variants.head()

,annotation,count,residue,variationId,scale,color
577,"614D>G(1153554), 614D>N(162), 614D>A(18)",1153734,614,"S:p.614D>G(1153554), S:p.614D>N(162), S:p.614D...",0.914755,#8e0912
638,"681P>H(363478), 681P>R(2312), 681P>L(464), 681...",366328,681,"S:p.681P>H(363478), S:p.681P>R(2312), S:p.681P...",0.839573,#ac1117
474,"501N>Y(350436), 501N>T(2846), 501N>S(24), 501N...",353324,501,"S:p.501N>Y(350436), S:p.501N>T(2846), S:p.501N...",0.837204,#ac1117
672,"716T>I(345552), 716T>A(58), 716T>K(2), 716T>S(2)",345614,716,"S:p.716T>I(345552), S:p.716T>A(58), S:p.716T>K...",0.835758,#ac1117
1046,"1118D>H(343072), 1118D>Y(698), 1118D>V(30), 11...",343824,1118,"S:p.1118D>H(343072), S:p.1118D>Y(698), QHD4341...",0.835418,#ac1117


### Get 3D structure for selected protein

In [30]:
# some Electron microscopy structures are of low resolution. Keep only high and medium resolution structures (0 - 4 Å).
resolution_threshold = 3.9

In [31]:
query = """
MATCH (g:Gene)-[:ENCODES]->(p:Protein{name: $protein_name, taxonomyId: $pathogen_taxonomy_id})-[h:HAS_TERTIARY_STRUCTURE]->(c:Chain)-[:IS_PART_OF_STRUCTURE]->(s:Structure)
WHERE s.resolution <= $resolution_threshold
RETURN p.name AS name, p.start, p.end, c.name AS structureChainId, c.uniprotStart, c.uniprotEnd, c.pdbStart, c.pdbEnd, s.resolution AS resolution, s.description AS description, h.coverage AS coverage
ORDER BY resolution, coverage DESC
"""

In [32]:
chains = graph.run(query, protein_name=protein_name, pathogen_taxonomy_id=pathogen_taxonomy_id, 
                   resolution_threshold=resolution_threshold).to_data_frame()

In [33]:
chains['structureChainId'] = chains['structureChainId'].str[4:]

In [34]:
chains['structureId'] = chains['structureChainId'].str[:4]

In [35]:
chains.query('structureId == "6XDG"')

,c.pdbEnd,c.pdbStart,c.uniprotEnd,c.uniprotStart,coverage,description,name,p.end,p.start,resolution,structureChainId,structureId
757,[526],[333],[526],[333],0.152396,"Spike protein S1, REGN10933 antibody Fab fragm...",Spike glycoprotein,1273,1,3.9,6XDG.E,6XDG


In [36]:
if with_antibodies:
    chains = chains[chains['description'].str.contains('heavy') | 
                    chains['description'].str.contains('antibody') |
                    chains['description'].str.contains('nanobody') |
                    chains['description'].str.contains('Fab')]

In [37]:
chains.query('structureId == "6XDG"')

,c.pdbEnd,c.pdbStart,c.uniprotEnd,c.uniprotStart,coverage,description,name,p.end,p.start,resolution,structureChainId,structureId
757,[526],[333],[526],[333],0.152396,"Spike protein S1, REGN10933 antibody Fab fragm...",Spike glycoprotein,1273,1,3.9,6XDG.E,6XDG


In [38]:
chains.head()

,c.pdbEnd,c.pdbStart,c.uniprotEnd,c.uniprotStart,coverage,description,name,p.end,p.start,resolution,structureChainId,structureId
0,[527],[324],[527],[324],0.160251,"Spike protein S1, the heavy chain of Fab fragm...",Spike glycoprotein,1273,1,1.400,7EAM.A,7EAM
1,[527],[324],[527],[324],0.160251,"Spike protein S1, the heavy chain of Fab fragm...",Spike glycoprotein,1273,1,1.400,7EAM.B,7EAM
3,"[359, 369, 384, 516]","[338, 366, 371, 392]","[359, 369, 384, 516]","[338, 366, 371, 392]",0.129615,"Spike protein S1, COVA2-39 heavy chain, COVA2-...",Spike glycoprotein,1273,1,1.712,7JMP.A,7JMP
4,[527],[334],[527],[334],0.152396,"LY-CoV488 Fab heavy chain, LY-CoV488 Fab light...",Spike glycoprotein,1273,1,1.720,7KMH.C,7KMH
5,[527],[334],[527],[334],0.152396,"LY-CoV481 Fab heavy chain, LY-CoV481 Fab light...",Spike glycoprotein,1273,1,1.730,7KMI.C,7KMI


In [39]:
chains.drop_duplicates(subset=['structureChainId'], inplace=True)
chains.sort_values(by=['structureChainId'], inplace=True)

#### Map uniprot residue numbers to PDB residue numbers

In [40]:
def uniprot_to_pdb_mapping(row):
    mapping = dict()
    for (us,ue, ps, pe) in zip(row['c.uniprotStart'], row['c.uniprotEnd'], row['c.pdbStart'], row['c.pdbEnd']):
        ps = int(ps)
        pe = int(pe)
        if (ue-us != pe-ps):
            print('length mismatch:', row['structureChainId'], ue-us, pe-ps)
        else:
            offset = ps - us
            for v in range(us, ue+1):
                mapping[v] = offset + v
                
    #print(mapping)
    return mapping

In [41]:
chains['mapping'] = chains.apply(lambda row: uniprot_to_pdb_mapping(row), axis=1)

In [42]:
chains.query('structureId == "6XDG"')

,c.pdbEnd,c.pdbStart,c.uniprotEnd,c.uniprotStart,coverage,description,name,p.end,p.start,resolution,structureChainId,structureId,mapping
757,[526],[333],[526],[333],0.152396,"Spike protein S1, REGN10933 antibody Fab fragm...",Spike glycoprotein,1273,1,3.9,6XDG.E,6XDG,"{333: 333, 334: 334, 335: 335, 336: 336, 337: ..."


### Visualize mutation sites

Mutations are mapped onto protein chains for available 3D protein structures.

Display options:

|||
|:-|:-|
| *show_bio_assembly* | Toggle display of the biologically relevant quaternary structure |
| *show_surface* | Toggle surface for protein chain |
| *show_short_label* | Toggle display of mutation information<br>{UniProtResidue}{aminoAcid1}>{aminoAcid2}(# observations)<br>Example: 501N>Y(350436)|
| *show_long_label* | Toggle display of mutation information<br>{PDBId}.{chainId}.{PDBResidue}: {geneName}.p{UniProtResidue}{aminoAcid1}>{aminoAcid2}(# observations)<br>Example: 6XDG.E.501: S:p.501N>Y(350436) |
| *size* | Change size of visualization |
| *font* | Change font size of annotations |
| *logFreq* | Change minimum threshold to display mutations based on normalized log of mutation frequency [0.0 - 1.0]|
| *structure* | Move slider to browse through available structures |

#### Example: Move the structure slider to PDB ID:6XDG to see how mutations (e.g., 501N>Y) effect the binding of the Regeneron antibodies 

In [43]:
# Setup viewer
def view_mutations(df, variants, *args):
    chainIds = list(df['structureChainId'])

    def view3d(show_bio_assembly, show_surface, show_short_label, show_long_label, size, font, logFreq, i): 
        pdb_id, chain_id = chainIds[i].split('.')
        global viewer1
        viewer1 = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly}, width=size, height=size)

        # polymer style
        viewer1.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.8}})

        # highlight chain of interest in blue
        viewer1.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue'}})
        
        # non-polymer style
        viewer1.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False, 'colorscheme': 'greenCarbon'}})
        
        mapping = df['mapping'].iloc[i]

        for row in variants.itertuples():
            # get PDB residue mapping from a UniProt residue number
            res_num = mapping.get(row.residue, 0)
            col = row.color
            if res_num > 0 and row.scale > logFreq:
                mut_res = {'resi': res_num, 'chain': chain_id}
                viewer1.addStyle(mut_res, {'sphere':{'color':col, 'opacity': 1.0}}) 

                if show_short_label:
                    label = row.annotation
                if show_long_label:
                    label = chainIds[i] + "." + str(res_num) + ": " + row.variationId
                if show_short_label or show_long_label:
                    viewer1.addLabel(label, {'fontSize':font,'fontColor': 'black','backgroundColor':'ivory', 'opacity': 1.0}, {'resi': res_num, 'chain': chain_id})

        description = df['description'].iloc[i]
        resolution = df['resolution'].iloc[i]
        coverage = df['coverage'].iloc[i]
        name = df['name'].iloc[i]
        
        print(name)
        print()
        print(f'PDB Id: {pdb_id}, chain Id: {chain_id}, resolution: {resolution}, sequence coverage: {coverage:.2f}')
        print(f'description: {description}')
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])

        viewer1.zoomTo({'chain': chain_id})
        viewer1.center({'chain': chain_id})
        
        if show_surface:
             viewer1.addSurface(py3Dmol.SES,{'opacity':0.8,'color':'lightblue'},{'chain': chain_id})

        return viewer1.show()
       
    f_widget = IntSlider(value=9, min=5, max=20, description='font size', continuous_update=False)
    z_widget = IntSlider(value=750, min=500, max=1200, description='size', continuous_update=False)
    s_widget = IntSlider(min=0, max=len(chainIds)-1, description='structure', continuous_update=False)
    l_widget = FloatSlider(value=0.8, min=0, max=1, step=0.05, description='logFreq:', 
                           continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f')
    
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_short_label=True, show_long_label=False, size=z_widget, font=f_widget, logFreq=l_widget, i=s_widget)

def view_image1():
    return viewer1.png()

In [44]:
view_mutations(chains, variants);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

In [45]:
# https://stackoverflow.com/questions/32468402/how-to-explode-a-list-inside-a-dataframe-cell-into-separate-rows
import copy

def pandas_explode(df, column_to_explode):
    """
    Similar to Hive's EXPLODE function, take a column with iterable elements, and flatten the iterable to one element 
    per observation in the output table

    :param df: A dataframe to explod
    :type df: pandas.DataFrame
    :param column_to_explode: 
    :type column_to_explode: str
    :return: An exploded data frame
    :rtype: pandas.DataFrame
    """

    # Create a list of new observations
    new_observations = list()

    # Iterate through existing observations
    for row in df.to_dict(orient='records'):

        # Take out the exploding iterable
        explode_values = row[column_to_explode]
        del row[column_to_explode]

        # Create a new observation for every entry in the exploding iterable & add all of the other columns
        for explode_value in explode_values.items():

            # Deep copy existing observation
            new_observation = copy.deepcopy(row)

            # Add one (newly flattened) value from exploding iterable
            new_observation[column_to_explode] = explode_value

            # Add to the list of new observations
            new_observations.append(new_observation)

    # Create a DataFrame
    return_df = pd.DataFrame(new_observations)

    # Return
    return return_df

### Expand chains into residues

In [46]:
residues = pandas_explode(chains, 'mapping')
residues['uniprotPosition'] = residues['mapping'].apply(lambda x: x[0])
residues['pdbPosition'] = residues['mapping'].apply(lambda x: x[1])
residues.drop(columns='mapping', inplace=True)

In [47]:
residues = residues.drop(columns=['c.pdbEnd', 'c.pdbStart', 'c.uniprotEnd', 'c.uniprotStart', 'p.end', 'p.start'])

In [48]:
residues.head()

,coverage,description,name,resolution,structureChainId,structureId,uniprotPosition,pdbPosition
0,0.153181,"CR3022 Fab heavy chain, CR3022 Fab light chain...",Spike glycoprotein,3.084,6W41.C,6W41,333,333
1,0.153181,"CR3022 Fab heavy chain, CR3022 Fab light chain...",Spike glycoprotein,3.084,6W41.C,6W41,334,334
2,0.153181,"CR3022 Fab heavy chain, CR3022 Fab light chain...",Spike glycoprotein,3.084,6W41.C,6W41,335,335
3,0.153181,"CR3022 Fab heavy chain, CR3022 Fab light chain...",Spike glycoprotein,3.084,6W41.C,6W41,336,336
4,0.153181,"CR3022 Fab heavy chain, CR3022 Fab light chain...",Spike glycoprotein,3.084,6W41.C,6W41,337,337


In [49]:
variants = variants[['residue', 'variationId', 'annotation', 'scale', 'color']]

In [50]:
residues_variants = residues.merge(variants, left_on='uniprotPosition', right_on='residue')

In [51]:
residues_variants.head()

,coverage,description,name,resolution,structureChainId,structureId,uniprotPosition,pdbPosition,residue,variationId,annotation,scale,color
0,0.153181,"CR3022 Fab heavy chain, CR3022 Fab light chain...",Spike glycoprotein,3.084,6W41.C,6W41,333,333,333,"S:p.333T>I(4), S:p.333T>K(4)","333T>I(4), 333T>K(4)",0.136274,#fedbcc
1,0.750196,"spike glycoprotein, S309 neutralizing antibody...",Spike glycoprotein,3.100,6WPS.A,6WPS,333,333,333,"S:p.333T>I(4), S:p.333T>K(4)","333T>I(4), 333T>K(4)",0.136274,#fedbcc
2,0.750196,"spike glycoprotein, S309 neutralizing antibody...",Spike glycoprotein,3.100,6WPS.B,6WPS,333,333,333,"S:p.333T>I(4), S:p.333T>K(4)","333T>I(4), 333T>K(4)",0.136274,#fedbcc
3,0.750196,"spike glycoprotein, S309 neutralizing antibody...",Spike glycoprotein,3.100,6WPS.E,6WPS,333,333,333,"S:p.333T>I(4), S:p.333T>K(4)","333T>I(4), 333T>K(4)",0.136274,#fedbcc
4,0.742341,"spike glycoprotein, S309 neutralizing antibody...",Spike glycoprotein,3.700,6WPT.A,6WPT,333,333,333,"S:p.333T>I(4), S:p.333T>K(4)","333T>I(4), 333T>K(4)",0.136274,#fedbcc


In [52]:
residues_variants.to_csv(output_file_name, index=False)

## Now run the next step
Map mutations occuring at protein-protein interaction sites: [2-MapToPolymerInteractions.ipynb](2-MapToPolymerInteractions.ipynb)